In [75]:

from flask import Flask, jsonify
from flask_restx import Api, Resource, fields, reqparse
import pickle
from watchdog.events import EVENT_TYPE_CREATED, EVENT_TYPE_OPENED
import pandas as pd


In [76]:
app = Flask(__name__)
api = Api(app, version='1.0', title='Modelo de Predicción de Precios de Autos',
          description='API para predecir el precio de un auto según sus características TEAM WPS')


In [77]:
# Procesamiento de los datos de Entrada
#Importación del modelo a utilizar 
model = pickle.load(open('modelo_entrenado.pkl', 'rb'))

pipeline = pickle.load(open('preprocessor.pkl', 'rb'))

# Creación del namespace para los endpoints
namespace = api.namespace('predict', description='Predicción de precios de vehiculos')

parser = api.parser()
parser.add_argument('Year', type=int, required=True, help='Modelo del Auto (Año)', location='args')
parser.add_argument('Mileage', type=int, required=True, help='Millas Recorridas', location='args')
parser.add_argument('State', type=str, required=True, help='Estado donde se encuentra el vehiculo', location='args')
parser.add_argument('Make', type=str, required=True, help='Marca', location='args')
parser.add_argument('Model', type=str, required=True, help='Modelo', location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})


@api.route('/predict')
class Predict(Resource):
    @api.doc(parser=parser)
    def get(self):
        # Parseamos los parámetros de entrada
        args = parser.parse_args()
        print(f'Argumentos recibidos: {args}')
        
        # Creamos una matriz con los valores de entrada
        input_data = [[args['Year'], args['Mileage'], args['State'], args['Make'], args['Model']]]
        df = pd.DataFrame(input_data, columns=['Year', 'Mileage', 'State', 'Make', 'Model'])
        print(f'DataFrame generado: \n{df}')
        
        # Transformamos los valores de entrada utilizando el preprocesador
        input_data_transformed = pipeline.transform(df)
        print(f'DataFrame transformado: \n{input_data_transformed}')
        
        
        # Realizamos la predicción utilizando el modelo entrenado
        predicted_price = model.predict(input_data_transformed)
        print(f'Precio predicho: {predicted_price[0]}')
        
        
        # Devolvemos la predicción como un diccionario JSON
        return jsonify({'predicted_price': predicted_price[0]})
        
    
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.80:5000
Press CTRL+C to quit
127.0.0.1 - - [01/May/2023 21:23:22] "GET /predict?Year=2010&Mileage=9913&State=FL&Make=Jeep&Model=Wrangler HTTP/1.1" 200 -


Argumentos recibidos: {'Year': 2010, 'Mileage': 9913, 'State': 'FL', 'Make': 'Jeep', 'Model': 'Wrangler'}
DataFrame generado: 
   Year  Mileage State  Make     Model
0  2010     9913    FL  Jeep  Wrangler
DataFrame transformado: 
  (0, 0)	-0.9692905216605613
  (0, 1)	-1.1044085958703047
  (0, 70)	1.0
  (0, 580)	1.0
Precio predicho: 29683.193223309034


127.0.0.1 - - [01/May/2023 21:23:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 21:23:29] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 21:23:29] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 21:23:29] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 21:23:29] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 21:23:30] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 21:23:30] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2023 21:23:49] "GET /predict?Year=2017&Mileage=9913&State=FL&Make=Jeep&Model=Wrangler HTTP/1.1" 200 -


Argumentos recibidos: {'Year': 2017, 'Mileage': 9913, 'State': 'FL', 'Make': 'Jeep', 'Model': 'Wrangler'}
DataFrame generado: 
   Year  Mileage State  Make     Model
0  2017     9913    FL  Jeep  Wrangler
DataFrame transformado: 
  (0, 0)	1.156306501219685
  (0, 1)	-1.1044085958703047
  (0, 70)	1.0
  (0, 580)	1.0
Precio predicho: 38381.72163895606
